# OpenAI Agents SDK - 멀티모달 비전

이 튜토리얼은 OpenAI Agents SDK에서 비전 기능을 활용하여 이미지를 분석하는 방법을 다룬다. 기본 이미지 분석부터 엔터프라이즈급 비전 에이전트 구축까지 멀티모달 AI 패턴을 학습한다.

In [ ]:
# 필요한 패키지 설치
!pip install openai-agents python-dotenv requests

---

## 1. 기본 이미지 비전 분석

이 챕터에서는 OpenAI Agents SDK로 이미지를 분석하는 기본 방법을 다룬다.

### 학습 내용

- 이미지를 base64로 인코딩하는 방법
- 비전 가능 에이전트 생성
- 올바른 입력 형식 (input_text, input_image)
- URL과 로컬 파일 모두 처리

### 환경 설정

In [2]:
import os
import asyncio
import base64
import requests
from dotenv import load_dotenv
from agents import Agent, Runner, ModelSettings

# 환경 변수 로드
load_dotenv()

MODEL = "gpt-4o-mini"

### 이미지 인코딩 유틸리티 함수

In [3]:
def encode_image_from_url(image_url: str) -> str:
    """
    URL에서 이미지를 다운로드하고 base64로 인코딩한다.
    
    Args:
        image_url: 이미지 URL
    
    Returns:
        base64 인코딩된 이미지 문자열
    """
    response = requests.get(image_url)
    response.raise_for_status()
    return base64.b64encode(response.content).decode('utf-8')


def encode_image_from_file(image_path: str) -> str:
    """
    로컬 이미지 파일을 base64로 인코딩한다.
    
    Args:
        image_path: 로컬 이미지 파일 경로
    
    Returns:
        base64 인코딩된 이미지 문자열
    """
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')


print("이미지 인코딩 유틸리티 함수가 정의되었다:")
print("  - encode_image_from_url: URL에서 이미지 인코딩")
print("  - encode_image_from_file: 로컬 파일에서 이미지 인코딩")

이미지 인코딩 유틸리티 함수가 정의되었다:
  - encode_image_from_url: URL에서 이미지 인코딩
  - encode_image_from_file: 로컬 파일에서 이미지 인코딩


### 올바른 비전 입력 형식

OpenAI Agents SDK에서 이미지를 분석하려면 특정 입력 형식을 사용해야 한다. GitHub issue #159에서 논의된 올바른 형식을 사용한다.

In [4]:
async def analyze_image_correct(image_url: str, question: str):
    """
    Agents SDK로 이미지를 분석하는 올바른 방법이다.
    
    Args:
        image_url: 이미지 URL 또는 파일 경로
        question: 이미지에 대한 질문
    
    Returns:
        분석 결과
    """
    
    # 비전 가능 에이전트 생성
    agent = Agent(
        name="Vision Agent",
        model="gpt-4o",  # 비전 가능 모델
        model_settings=ModelSettings(temperature=0.1, max_tokens=1024),
        instructions="""
        당신은 이미지를 보고 분석할 수 있습니다.
        관찰한 내용을 정확하게 설명하고 이미지에 대한 질문에 답하세요.
        한국어로 응답하세요.
        """
    )
    
    # base64 이미지 가져오기
    if image_url.startswith('http'):
        base64_image = encode_image_from_url(image_url)
        image_data_url = f"data:image/jpeg;base64,{base64_image}"
    else:
        # 파일 경로로 가정
        base64_image = encode_image_from_file(image_url)
        image_data_url = f"data:image/jpeg;base64,{base64_image}"
    
    # 올바른 형식 - 별도의 텍스트와 이미지 컴포넌트로 구조화된 입력 사용
    result = await Runner.run(agent, input=[
        {
            "role": "user",
            "content": [
                {"type": "input_text", "text": question},
                {"type": "input_image", "image_url": image_data_url}
            ]
        }
    ])
    
    return result.final_output


print("analyze_image_correct 함수가 정의되었다.")
print("\n핵심 포인트:")
print("  1. model='gpt-4o' - 비전 가능 모델 사용")
print("  2. input_text와 input_image 타입 사용")
print("  3. 이미지는 data URL 형식으로 전달")

analyze_image_correct 함수가 정의되었다.

핵심 포인트:
  1. model='gpt-4o' - 비전 가능 모델 사용
  2. input_text와 input_image 타입 사용
  3. 이미지는 data URL 형식으로 전달


### 기본 비전 분석 데모

In [5]:
async def demo_basic_vision():
    """
    기본 비전 분석을 시연한다.
    """
    
    print("🖼️ 기본 비전 분석 데모")
    print("OpenAI Agents SDK의 올바른 입력 형식 사용\n")
    
    # 테스트 이미지 1: 제품 이미지
    test_image_1 = "https://images.unsplash.com/photo-1586953208448-b95a79798f07?w=600"
    
    print("=" * 60)
    print("테스트 1: 제품 이미지 분석")
    print(f"이미지 URL: {test_image_1}")
    print("=" * 60)
    
    result1 = await analyze_image_correct(
        image_url=test_image_1,
        question="이 이미지에서 무엇을 볼 수 있나요? 색상과 보이는 텍스트를 설명해주세요."
    )
    
    print(f"\n🔍 분석 결과:")
    print(result1)
    print("=" * 60)
    
    # 테스트 이미지 2: 차량 이미지
    print("\n" + "-" * 60 + "\n")
    
    test_image_2 = "https://images.unsplash.com/photo-1560472354-b33ff0c44a43?w=400"
    
    print("=" * 60)
    print("테스트 2: 차량 이미지 분석")
    print(f"이미지 URL: {test_image_2}")
    print("=" * 60)
    
    result2 = await analyze_image_correct(
        image_url=test_image_2,
        question="이 차량을 설명해주세요 - 종류, 색상, 눈에 띄는 특징을 알려주세요."
    )
    
    print(f"\n🚗 분석 결과:")
    print(result2)
    print("=" * 60)

# 데모 실행
await demo_basic_vision()

🖼️ 기본 비전 분석 데모
OpenAI Agents SDK의 올바른 입력 형식 사용

테스트 1: 제품 이미지 분석
이미지 URL: https://images.unsplash.com/photo-1586953208448-b95a79798f07?w=600

🔍 분석 결과:
이미지에는 스마트폰이 책상 위에 놓여 있습니다. 스마트폰 화면에는 여러 개의 이미지가 배열되어 있으며, 이는 소셜 미디어나 이미지 공유 플랫폼의 피드처럼 보입니다. 화면 상단에는 "For you"라는 텍스트가 보입니다. 스마트폰의 테두리는 검은색이며, 배경은 밝은 회색입니다. 주변에 노트북의 일부분도 보입니다. 전반적으로 깔끔하고 현대적인 느낌을 줍니다.

------------------------------------------------------------

테스트 2: 차량 이미지 분석
이미지 URL: https://images.unsplash.com/photo-1560472354-b33ff0c44a43?w=400

🚗 분석 결과:
이 이미지는 차량이 아니라 웹 분석 대시보드의 스크린샷입니다. 화면에는 총 클릭 수, 총 노출 수, 평균 클릭률(CTR), 평균 위치 등의 데이터가 표시되어 있습니다. 그래프는 시간에 따른 데이터의 변화를 시각적으로 보여주고 있습니다.


### 여러 이미지 순차 분석

In [9]:
async def analyze_multiple_images():
    """
    여러 이미지를 순차적으로 분석한다.
    """
    
    images = [
        {
            "url": "https://images.unsplash.com/photo-1521791136064-7986c2920216?w=600",
            "question": "이 사무실 공간의 분위기와 특징을 설명해주세요."
        },
        {
            "url": "https://images.unsplash.com/photo-1560472354-b33ff0c44a43?w=600",
            "question": "이 이미지에서 어떤 텍스트나 숫자가 보이나요?"
        }
    ]
    
    print("🖼️ 다중 이미지 분석 데모")
    print("=" * 60)
    
    for i, img in enumerate(images, 1):
        print(f"\n📷 이미지 {i}:")
        print(f"URL: {img['url']}")
        print(f"질문: {img['question']}")
        print("-" * 40)
        
        try:
            result = await analyze_image_correct(img['url'], img['question'])
            print(f"결과: {result}")
        except Exception as e:
            print(f"에러: {e}")
        
        print("-" * 40)

# 다중 이미지 분석 실행
await analyze_multiple_images()

🖼️ 다중 이미지 분석 데모

📷 이미지 1:
URL: https://images.unsplash.com/photo-1521791136064-7986c2920216?w=600
질문: 이 사무실 공간의 분위기와 특징을 설명해주세요.
----------------------------------------
결과: 이미지에는 두 사람이 악수를 나누고 있는 장면이 중심에 있습니다. 배경으로 보아 사무실이나 회의실 같은 공간으로 보입니다. 책상과 의자가 배치되어 있으며, 벽에는 액자가 걸려 있습니다. 전체적으로 깔끔하고 전문적인 분위기를 주며, 협력이나 비즈니스 미팅이 이루어지는 상황을 연상시킵니다.
----------------------------------------

📷 이미지 2:
URL: https://images.unsplash.com/photo-1560472354-b33ff0c44a43?w=600
질문: 이 이미지에서 어떤 텍스트나 숫자가 보이나요?
----------------------------------------
결과: 이미지에는 다음과 같은 텍스트와 숫자가 보입니다:

- "Total clicks: 223"
- "Total impressions: 17.6K"
- "Average CTR: 1.3%"
- "Average position: 25.2"

또한, 그래프가 있으며 날짜는 "2/24/19"부터 "6/2/19"까지 표시되어 있습니다.
----------------------------------------


---

## 고급 비전 에이전트

이 챕터에서는 도구, 관찰 가능성, 비즈니스 메트릭을 포함한 엔터프라이즈급 비전 에이전트를 구축한다.

### 학습 내용

- 비전 메트릭 클래스 설계
- 이미지 처리 도구 구현
- 엔터프라이즈 비전 에이전트 래퍼
- 품질 평가 및 에스컬레이션

### 비전 메트릭 클래스

In [10]:
import time
import random
import json
import logging
from datetime import datetime
from dataclasses import dataclass, field
from typing import Dict, List, Optional, Any
from collections import defaultdict
from urllib.parse import urlparse
from agents import function_tool

In [11]:
@dataclass
class VisionMetrics:
    """
    비전/이미지 분석 작업에 특화된 메트릭 클래스이다.
    """
    image_url: str = ""                    # 이미지 URL
    image_size_bytes: int = 0              # 이미지 크기 (바이트)
    image_dimensions: str = ""             # 이미지 해상도
    image_format: str = ""                 # 이미지 형식
    analysis_type: str = ""                # 분석 유형
    confidence_score: float = 0.0          # 신뢰도 점수 (0-10)
    objects_detected: int = 0              # 감지된 객체 수
    text_extracted: bool = False           # 텍스트 추출 여부
    processing_time: float = 0.0           # 처리 시간 (초)


@dataclass
class AgentPerformanceMetrics:
    """
    비전 작업을 포함한 향상된 성능 메트릭 클래스이다.
    """
    operation_name: str                                      # 작업 이름
    start_time: float                                        # 시작 시간
    end_time: float = 0.0                                    # 종료 시간
    token_usage: Dict[str, int] = field(default_factory=dict)  # 토큰 사용량
    tool_calls: List[str] = field(default_factory=list)       # 도구 호출
    reasoning_depth: int = 0                                  # 추론 깊이
    vision_metrics: Optional[VisionMetrics] = None            # 비전 메트릭
    
    @property
    def duration(self) -> float:
        if self.end_time > 0:
            return self.end_time - self.start_time
        return time.time() - self.start_time


@dataclass  
class BusinessMetrics:
    """
    비전 작업용 비즈니스 메트릭 클래스이다.
    """
    customer_satisfaction_score: float = 0.0   # 고객 만족도
    resolution_time: float = 0.0               # 해결 시간
    escalation_required: bool = False          # 에스컬레이션 필요
    cost_per_interaction: float = 0.0          # 상호작용당 비용
    accuracy_confidence: float = 0.0           # AI 분석 신뢰도
    human_review_needed: bool = False          # 사람 검토 필요


print("비전 메트릭 클래스가 정의되었다:")
print("  - VisionMetrics: 이미지 분석 메트릭")
print("  - AgentPerformanceMetrics: 성능 메트릭")
print("  - BusinessMetrics: 비즈니스 메트릭")

비전 메트릭 클래스가 정의되었다:
  - VisionMetrics: 이미지 분석 메트릭
  - AgentPerformanceMetrics: 성능 메트릭
  - BusinessMetrics: 비즈니스 메트릭


### 비전 메트릭 수집기

In [12]:
class VisionMetricsCollector:
    """
    비전 작업을 위한 메트릭 수집기 클래스이다.
    """
    
    def __init__(self):
        self.metrics: List[BusinessMetrics] = []
        self.vision_stats: List[VisionMetrics] = []
        self.operation_stats = defaultdict(lambda: {
            'total_count': 0,
            'success_count': 0,
            'total_duration': 0.0,
            'total_cost': 0.0,
            'avg_confidence': 0.0
        })
    
    def record_interaction(self, operation_type: str, metrics: BusinessMetrics, 
                          vision_metrics: VisionMetrics = None):
        """비전 메트릭과 함께 비즈니스 상호작용을 기록한다."""
        self.metrics.append(metrics)
        if vision_metrics:
            self.vision_stats.append(vision_metrics)
        
        stats = self.operation_stats[operation_type]
        stats['total_count'] += 1
        stats['total_duration'] += metrics.resolution_time
        stats['total_cost'] += metrics.cost_per_interaction
        
        if vision_metrics:
            stats['avg_confidence'] = (
                (stats['avg_confidence'] * (stats['total_count'] - 1) + 
                 vision_metrics.confidence_score) / stats['total_count']
            )
        
        if not metrics.escalation_required:
            stats['success_count'] += 1
    
    def get_summary(self) -> Dict[str, Any]:
        """비전 분석을 포함한 요약을 생성한다."""
        if not self.metrics:
            return {}
            
        total_interactions = len(self.metrics)
        avg_satisfaction = sum(m.customer_satisfaction_score for m in self.metrics) / total_interactions
        avg_resolution_time = sum(m.resolution_time for m in self.metrics) / total_interactions
        escalation_rate = sum(1 for m in self.metrics if m.escalation_required) / total_interactions
        
        summary = {
            'total_interactions': total_interactions,
            'average_satisfaction': round(avg_satisfaction, 2),
            'average_resolution_time': round(avg_resolution_time, 2),
            'escalation_rate': round(escalation_rate * 100, 1),
            'operation_breakdown': dict(self.operation_stats)
        }
        
        # 비전별 메트릭 추가
        if self.vision_stats:
            avg_confidence = sum(v.confidence_score for v in self.vision_stats) / len(self.vision_stats)
            total_objects = sum(v.objects_detected for v in self.vision_stats)
            text_extraction_rate = sum(1 for v in self.vision_stats if v.text_extracted) / len(self.vision_stats)
            
            summary['vision_analytics'] = {
                '총_분석_이미지': len(self.vision_stats),
                '평균_신뢰도': round(avg_confidence, 2),
                '총_감지_객체': total_objects,
                '텍스트_추출_비율': round(text_extraction_rate * 100, 1)
            }
        
        return summary


print("VisionMetricsCollector 클래스가 정의되었다.")

VisionMetricsCollector 클래스가 정의되었다.


### 비전 도구 정의

In [13]:
@function_tool
def fetch_and_analyze_image(image_url: str, analysis_type: str = "general") -> str:
    """
    온라인 이미지를 가져와 분석을 위해 준비한다.
    
    Args:
        image_url: 이미지 URL
        analysis_type: 분석 유형 (general, damage, quality 등)
    
    Returns:
        이미지 가져오기 결과
    """
    try:
        # URL 유효성 검사
        parsed_url = urlparse(image_url)
        if not parsed_url.scheme or not parsed_url.netloc:
            return f"잘못된 URL 형식: {image_url}"
        
        # 타임아웃과 함께 이미지 가져오기
        response = requests.get(image_url, timeout=10, headers={
            'User-Agent': 'AI-Vision-Agent/1.0'
        })
        response.raise_for_status()
        
        # 이미지 메타데이터 가져오기
        content_type = response.headers.get('content-type', 'unknown')
        image_size = len(response.content)
        
        # 기본 유효성 검사
        if image_size > 20 * 1024 * 1024:  # 20MB 제한
            return f"이미지가 너무 큼: {image_size} 바이트 (최대 20MB)"
        
        if not content_type.startswith('image/'):
            return f"잘못된 콘텐츠 타입: {content_type}"
        
        return f"이미지가 {analysis_type} 분석을 위해 준비됨. 크기: {image_size} 바이트, 타입: {content_type}"
        
    except requests.RequestException as e:
        return f"{image_url}에서 이미지 가져오기 실패: {str(e)}"
    except Exception as e:
        return f"이미지 처리 에러: {str(e)}"


@function_tool
def validate_image_quality(image_url: str) -> str:
    """
    이미지 품질과 기술 사양을 검증한다.
    
    Args:
        image_url: 이미지 URL
    
    Returns:
        품질 검증 결과
    """
    try:
        response = requests.get(image_url, timeout=10)
        response.raise_for_status()
        
        image_size = len(response.content)
        content_type = response.headers.get('content-type', 'unknown')
        
        quality_score = random.uniform(6.5, 9.5)  # 품질 평가 시뮬레이션
        
        if image_size < 50 * 1024:  # 50KB 미만
            quality_score *= 0.8
            quality_note = "저해상도 감지"
        elif image_size > 5 * 1024 * 1024:  # 5MB 초과
            quality_score *= 0.9
            quality_note = "매우 고해상도"
        else:
            quality_note = "적절한 해상도"
        
        return f"이미지 품질 점수: {quality_score:.1f}/10. {quality_note}. 크기: {image_size} 바이트, 형식: {content_type}"
        
    except Exception as e:
        return f"이미지 품질 검증 실패: {str(e)}"


@function_tool
def escalate_vision_analysis(image_url: str, issue_type: str, 
                            confidence_threshold: float = 0.7) -> str:
    """
    신뢰도가 낮을 때 비전 분석을 사람 검토로 에스컬레이션한다.
    
    Args:
        image_url: 이미지 URL
        issue_type: 문제 유형
        confidence_threshold: 신뢰도 임계값
    
    Returns:
        에스컬레이션 확인 메시지
    """
    ticket_number = random.randint(1000, 9999)
    return f"{image_url}의 비전 분석이 사람 검토로 에스컬레이션됨. 문제: {issue_type}. 티켓 #VIS-{ticket_number} 생성됨."


print("비전 도구가 정의되었다:")
print("  - fetch_and_analyze_image: 이미지 가져오기")
print("  - validate_image_quality: 품질 검증")
print("  - escalate_vision_analysis: 에스컬레이션")

비전 도구가 정의되었다:
  - fetch_and_analyze_image: 이미지 가져오기
  - validate_image_quality: 품질 검증
  - escalate_vision_analysis: 에스컬레이션


### 엔터프라이즈 비전 에이전트 클래스

In [14]:
class EnterpriseVisionAgent:
    """
    종합적인 관찰 가능성을 갖춘 엔터프라이즈 비전 에이전트이다.
    온라인 이미지 분석과 비즈니스 메트릭, 구조화된 로깅을 처리한다.
    """
    
    def __init__(self):
        self.technical_metrics: List[AgentPerformanceMetrics] = []
        self.business_collector = VisionMetricsCollector()
        
        # 비전 가능 에이전트 생성
        self.agent = Agent(
            name="Enterprise Vision Analyst",
            instructions="""
            당신은 엔터프라이즈 운영을 위한 고급 AI 비전 분석가입니다.
            
            가능한 기능:
            - URL에서 온라인 이미지 분석
            - 품질 평가 및 손상 감지
            - 텍스트 추출 및 객체 식별
            - 제품 분석 및 준수 확인
            - 안전 및 보안 평가
            
            이미지 분석 시:
            1. 관찰한 내용의 상세 설명 제공
            2. 핵심 객체, 텍스트, 패턴 식별
            3. 품질, 손상, 준수 문제 평가
            4. 분석에 대한 신뢰도 점수 제공
            5. 필요시 에스컬레이션 권장
            
            항상 철저하고 정확하며 실행 가능한 인사이트를 제공하세요.
            한국어로 응답하세요.
            """,
            model="gpt-4o",
            tools=[fetch_and_analyze_image, validate_image_quality, escalate_vision_analysis]
        )
        
        # 비전 작업 비용
        self.operation_costs = {
            'vision_analysis': 0.008,
            'image_fetch': 0.001,
            'quality_check': 0.002,
            'escalation': 0.75,
            'token_cost_multiplier': 1.5
        }
        
        print(f"✓ '{self.agent.name}' 에이전트 초기화 완료")
    
    async def analyze_image(self, image_url: str, analysis_request: str,
                           operation_type: str = "image_analysis",
                           customer_id: str = None) -> Dict:
        """
        종합적인 관찰 가능성과 함께 온라인 이미지를 분석한다.
        """
        
        session_id = f"vision_{int(time.time() * 1000)}"
        customer_id = customer_id or f"cust_{random.randint(1000, 9999)}"
        
        # 메트릭 초기화
        tech_metrics = AgentPerformanceMetrics(
            operation_name=f"vision_analysis_{operation_type}",
            start_time=time.time()
        )
        
        business_metrics = BusinessMetrics()
        vision_metrics = VisionMetrics(
            image_url=image_url,
            analysis_type=operation_type
        )
        
        try:
            print(f"🖼️  비전 분석 시작: {analysis_request}")
            print(f"🔗 이미지 URL: {image_url}")
            print(f"📊 세션: {session_id} | 고객: {customer_id}")
            print(f"⏱️  시작: {time.strftime('%H:%M:%S', time.localtime(tech_metrics.start_time))}")
            
            # 비전 프롬프트 구성
            vision_prompt = f"""
            이 온라인 이미지를 분석하세요: {image_url}
            
            고객 요청: {analysis_request}
            
            다음을 제공하세요:
            1. 보이는 내용의 상세 설명
            2. 식별된 핵심 객체와 요소
            3. 이미지에 있는 텍스트
            4. 품질 평가 (1-10 척도)
            5. 분석 신뢰도 수준 (1-10 척도)
            6. 발견된 문제, 손상, 우려 사항
            7. 권장 조치
            
            먼저 사용 가능한 도구를 사용하여 이미지를 가져오고 검증하세요.
            """
            
            # 비전 분석 실행
            result = await Runner.run(self.agent, vision_prompt)
            
            # 기술적 추적 완료
            tech_metrics.end_time = time.time()
            business_metrics.resolution_time = tech_metrics.duration
            vision_metrics.processing_time = tech_metrics.duration
            
            # 메트릭 계산
            vision_metrics = self._calculate_vision_metrics(vision_metrics, result)
            business_metrics = self._calculate_business_metrics(
                business_metrics, tech_metrics, vision_metrics, result
            )
            
            # 메트릭 기록
            tech_metrics.vision_metrics = vision_metrics
            self.technical_metrics.append(tech_metrics)
            self.business_collector.record_interaction(operation_type, business_metrics, vision_metrics)
            
            # 결과 표시
            self._display_results(tech_metrics, business_metrics, vision_metrics, result, session_id)
            
            return {
                'result': result,
                'session_id': session_id,
                'technical_metrics': tech_metrics,
                'business_metrics': business_metrics,
                'vision_metrics': vision_metrics
            }
            
        except Exception as e:
            tech_metrics.end_time = time.time()
            business_metrics.escalation_required = True
            business_metrics.human_review_needed = True
            print(f"❌ 비전 분석 에러: {e}")
            raise
    
    def _calculate_vision_metrics(self, vision_metrics: VisionMetrics, result) -> VisionMetrics:
        """분석 결과에서 비전별 메트릭을 계산한다."""
        
        result_text = result.final_output.lower()
        
        # 신뢰도 점수 추출 시뮬레이션
        if 'confidence' in result_text or '신뢰' in result_text:
            vision_metrics.confidence_score = random.uniform(7.0, 9.5)
        else:
            vision_metrics.confidence_score = random.uniform(6.0, 8.0)
        
        # 감지된 객체 추정
        object_keywords = ['object', 'item', 'element', '객체', '요소', '아이템']
        vision_metrics.objects_detected = sum(1 for kw in object_keywords if kw in result_text)
        
        # 텍스트 추출 확인
        vision_metrics.text_extracted = any(
            phrase in result_text for phrase in ['text', 'words', '텍스트', '글자', '문자']
        )
        
        # 이미지 메타데이터 시뮬레이션
        vision_metrics.image_size_bytes = random.randint(100000, 5000000)
        vision_metrics.image_dimensions = f"{random.randint(800, 2400)}x{random.randint(600, 1800)}"
        vision_metrics.image_format = random.choice(['JPEG', 'PNG', 'WebP'])
        
        return vision_metrics
    
    def _calculate_business_metrics(self, business_metrics: BusinessMetrics,
                                   tech_metrics: AgentPerformanceMetrics,
                                   vision_metrics: VisionMetrics,
                                   result) -> BusinessMetrics:
        """비전 작업의 비즈니스 메트릭을 계산한다."""
        
        # 비용 계산
        cost = self.operation_costs['vision_analysis']
        cost += len(tech_metrics.tool_calls) * self.operation_costs['image_fetch']
        
        # 낮은 신뢰도시 사람 검토 필요
        if vision_metrics.confidence_score < 7.0:
            business_metrics.human_review_needed = True
            cost += self.operation_costs['escalation']
        
        # 에스컬레이션 키워드 확인
        escalation_keywords = ['escalate', 'human', 'review', '에스컬레이션', '검토']
        if any(kw in result.final_output.lower() for kw in escalation_keywords):
            business_metrics.escalation_required = True
            cost += self.operation_costs['escalation']
        
        # 만족도 계산
        base_satisfaction = 3.0
        confidence_bonus = (vision_metrics.confidence_score - 5.0) * 0.4
        speed_bonus = max(0, (10.0 - tech_metrics.duration) * 0.1)
        satisfaction = min(5.0, base_satisfaction + confidence_bonus + speed_bonus)
        
        business_metrics.cost_per_interaction = round(cost, 4)
        business_metrics.customer_satisfaction_score = round(satisfaction, 1)
        business_metrics.accuracy_confidence = vision_metrics.confidence_score
        
        return business_metrics
    
    def _display_results(self, tech_metrics: AgentPerformanceMetrics,
                        business_metrics: BusinessMetrics,
                        vision_metrics: VisionMetrics, result, session_id: str):
        """종합 비전 분석 결과를 표시한다."""
        print("\n" + "="*75)
        print("🖼️  엔터프라이즈 비전 분석 대시보드")
        print(f"🔗 세션: {session_id}")
        print("="*75)
        
        # 비전 분석 결과
        print("\n👁️  비전 분석 결과")
        print("-" * 35)
        print(f"🎯 신뢰도 점수: {vision_metrics.confidence_score:.1f}/10")
        print(f"🔍 감지된 객체: {vision_metrics.objects_detected}")
        print(f"📝 텍스트 추출: {'예' if vision_metrics.text_extracted else '아니오'}")
        print(f"📐 이미지 해상도: {vision_metrics.image_dimensions}")
        print(f"💾 이미지 크기: {vision_metrics.image_size_bytes:,} 바이트")
        print(f"🖼️  형식: {vision_metrics.image_format}")
        
        # 기술적 성능
        print(f"\n🔧 기술적 성능")
        print("-" * 35)
        print(f"⏱️  처리 시간: {tech_metrics.duration:.2f}초")
        print(f"🛠️  사용된 도구: {len(tech_metrics.tool_calls)}개")
        
        # 비즈니스 영향
        print(f"\n💼 비즈니스 영향")
        print("-" * 35)
        print(f"😊 만족도 점수: {business_metrics.customer_satisfaction_score}/5.0")
        print(f"💰 분석 비용: ${business_metrics.cost_per_interaction}")
        print(f"👨‍💼 사람 검토: {'필요' if business_metrics.human_review_needed else '불필요'}")
        print(f"🆘 에스컬레이션: {'필요' if business_metrics.escalation_required else '불필요'}")
        
        # 분석 출력
        print(f"\n🤖 분석 결과")
        print("-" * 35)
        output = result.final_output
        print(f"{output[:500]}..." if len(output) > 500 else output)
        print("="*75)
    
    def get_dashboard(self) -> Dict:
        """비전 분석 대시보드를 생성한다."""
        return self.business_collector.get_summary()


print("EnterpriseVisionAgent 클래스가 정의되었다.")

EnterpriseVisionAgent 클래스가 정의되었다.


### 고급 비전 분석 데모

In [15]:
async def demo_enterprise_vision():
    """
    온라인 이미지 분석과 함께 비전 가능 에이전트를 시연한다.
    """
    
    # 비전 에이전트 초기화
    vision_agent = EnterpriseVisionAgent()
    
    # 테스트 시나리오
    vision_scenarios = [
        {
            "image_url": "https://images.unsplash.com/photo-1560472354-b33ff0c44a43?w=800",
            "request": "이 자동차 이미지에서 손상이나 문제가 있는지 분석해주세요",
            "operation_type": "damage_assessment",
            "customer_id": "cust_auto_001"
        },
        {
            "image_url": "https://images.unsplash.com/photo-1586953208448-b95a79798f07?w=800",
            "request": "이 제품 패키지의 품질과 상태를 평가해주세요",
            "operation_type": "quality_control",
            "customer_id": "cust_retail_002"
        }
    ]
    
    print("🚀 엔터프라이즈 비전 AI 에이전트 시연")
    print("=" * 75)
    print("기능:")
    print("• URL에서 온라인 이미지 분석")
    print("• 비전 및 텍스트 처리를 포함한 멀티모달 AI")
    print("• 종합적인 비즈니스 및 기술 관찰 가능성")
    print("• 자동 품질 평가 및 에스컬레이션")
    print("=" * 75)
    
    # 비전 분석 시나리오 실행
    results = []
    for i, scenario in enumerate(vision_scenarios, 1):
        print(f"\n🎯 비전 시나리오 {i}/{len(vision_scenarios)}")
        print(f"🔗 이미지: {scenario['image_url']}")
        print(f"📝 요청: {scenario['request']}")
        print(f"🏷️  유형: {scenario['operation_type']}")
        
        try:
            result = await vision_agent.analyze_image(
                image_url=scenario['image_url'],
                analysis_request=scenario['request'],
                operation_type=scenario['operation_type'],
                customer_id=scenario['customer_id']
            )
            results.append(result)
            
        except Exception as e:
            print(f"❌ 시나리오 {i} 에러: {e}")
        
        await asyncio.sleep(2)
    
    # 비전 분석 대시보드 생성
    print("\n" + "="*75)
    print("📊 비전 분석 대시보드")
    print("="*75)
    
    summary = vision_agent.get_dashboard()
    
    if summary:
        print(f"\n📈 성능 메트릭")
        print(f"총 분석: {summary.get('total_interactions', 0)}")
        print(f"평균 해결 시간: {summary.get('average_resolution_time', 0)}초")
        print(f"고객 만족도: {summary.get('average_satisfaction', 0)}/5.0")
        print(f"에스컬레이션 비율: {summary.get('escalation_rate', 0)}%")
        
        vision_analytics = summary.get('vision_analytics', {})
        if vision_analytics:
            print(f"\n🖼️  비전 인사이트")
            print(f"분석된 이미지: {vision_analytics.get('총_분석_이미지', 0)}")
            print(f"평균 신뢰도: {vision_analytics.get('평균_신뢰도', 0)}/10")
            print(f"텍스트 추출 비율: {vision_analytics.get('텍스트_추출_비율', 0)}%")

# 데모 실행
await demo_enterprise_vision()

✓ 'Enterprise Vision Analyst' 에이전트 초기화 완료
🚀 엔터프라이즈 비전 AI 에이전트 시연
기능:
• URL에서 온라인 이미지 분석
• 비전 및 텍스트 처리를 포함한 멀티모달 AI
• 종합적인 비즈니스 및 기술 관찰 가능성
• 자동 품질 평가 및 에스컬레이션

🎯 비전 시나리오 1/2
🔗 이미지: https://images.unsplash.com/photo-1560472354-b33ff0c44a43?w=800
📝 요청: 이 자동차 이미지에서 손상이나 문제가 있는지 분석해주세요
🏷️  유형: damage_assessment
🖼️  비전 분석 시작: 이 자동차 이미지에서 손상이나 문제가 있는지 분석해주세요
🔗 이미지 URL: https://images.unsplash.com/photo-1560472354-b33ff0c44a43?w=800
📊 세션: vision_1765712838111 | 고객: cust_auto_001
⏱️  시작: 20:47:18

🖼️  엔터프라이즈 비전 분석 대시보드
🔗 세션: vision_1765712838111

👁️  비전 분석 결과
-----------------------------------
🎯 신뢰도 점수: 8.2/10
🔍 감지된 객체: 2
📝 텍스트 추출: 예
📐 이미지 해상도: 1793x1166
💾 이미지 크기: 2,396,964 바이트
🖼️  형식: WebP

🔧 기술적 성능
-----------------------------------
⏱️  처리 시간: 5.15초
🛠️  사용된 도구: 0개

💼 비즈니스 영향
-----------------------------------
😊 만족도 점수: 4.7/5.0
💰 분석 비용: $0.008
👨‍💼 사람 검토: 불필요
🆘 에스컬레이션: 불필요

🤖 분석 결과
-----------------------------------
1. **보이는 내용의 상세 설명**:
   - 이미지에는 밝은 날씨 아래 주차된 빨간색 스포츠카가 보입니다. 차의 외형은 매끈하며 

---

## 멀티모달 비전 요약

1. 기본 비전 분석
   - gpt-4o 모델 사용 (비전 가능)
   - 이미지를 base64로 인코딩
   - input_text와 input_image 타입 사용
   -  URL과 로컬 파일 모두 지원

2. 비전 메트릭
   -  VisionMetrics: 이미지 분석 메트릭
   -  confidence_score: 신뢰도 점수
   -  objects_detected: 감지된 객체
   -  text_extracted: 텍스트 추출 여부

3. 비전 도구
   -  fetch_and_analyze_image: 이미지 가져오기
   -  validate_image_quality: 품질 검증
   -  escalate_vision_analysis: 에스컬레이션

4. 엔터프라이즈 비전 에이전트
   -  종합적인 관찰 가능성
   -  비즈니스 메트릭 추적
   -  자동 품질 평가
   -  낮은 신뢰도시 에스컬레이션

---

## 마무리

이 튜토리얼에서는 OpenAI Agents SDK의 멀티모달 비전 기능을 다루었다. 다음과 같은 내용을 학습하였다:

1. **기본 비전 분석**
   - 이미지 인코딩 유틸리티 (URL, 파일)
   - 올바른 입력 형식 (input_text, input_image)
   - gpt-4o-mini 비전 모델 사용

2. **고급 비전 에이전트**
   - VisionMetrics, BusinessMetrics 클래스
   - VisionMetricsCollector
   - 비전 도구 (가져오기, 품질 검증, 에스컬레이션)
   - EnterpriseVisionAgent 클래스

이러한 패턴을 활용하면 이미지 분석, 품질 검사, 손상 평가 등 다양한 비전 기반 애플리케이션을 구축할 수 있다.